In [1]:
from scholarly import scholarly, ProxyGenerator
from fp.fp import FreeProxy
import requests
import mysql.connector
import textdistance
import time
import random
import xmltodict
import urllib, urllib.request
from webcrawler.researchgate import IeeeCrawler, ResearchGateCrawler, SpringerCrawler, PubMedCrawler
import selenium as se

In [2]:
cnx = mysql.connector.connect(user='david', password='daviddung1993',
                              host='127.0.0.1',
                              database='computervision')
cursor = cnx.cursor()
headers = {"x-api-key": "M7HSjQNeTfai6l7JUiDZB8XYc85BHnHt3R0NXSEd"}
#pg = ProxyGenerator()

In [3]:
research_crawler = ResearchGateCrawler()
springer_crawler = SpringerCrawler()
pubmed_crawler = PubMedCrawler()

In [7]:
se.webdriver.ChromeOptions()

In [4]:
doi = research_crawler.extract_doi("Bayesian semi-parametric ROC analysis", "Erkanli")
pubmed_crawler.make_request(doi)

''

In [7]:
doi

'10.1002/sim.2496'

In [9]:
done = []

In [ ]:
done_file =  open("rg_done.txt","r")

lines_done = done_file.readlines()
for idx, line in enumerate(lines_done):
    try:
        if idx % 10 == 10:
            print(idx)
        manual =  open("manual.txt","r+")
        title = line[:-1]
        if title in done:
            continue
        print(title)
        done.append(title)
        cursor.execute("select distinct a.AuthorID, a.Name from Papers p,authoredBy b,Authors a where p.Title=%s and p.PaperID = b.PaperID and b.AuthoredByID not in (SELECT AuthorID from affiliatedTo) and b.AuthoredByID = a.AuthorID",[title])
        authors = cursor.fetchall()
        if not authors:
            continue
        first_name = authors[0][1].split()[-1]
        doi = research_crawler.extract_doi(title,first_name)
        affiliations = springer_crawler.make_request(doi)
        time.sleep(1)
        if not affiliations:
            affiliations = pubmed_crawler.make_request(doi)
            if not affiliations:
                continue

        for affiliation in affiliations:
            manual.write(":".join(affiliation))
            manual.close()
    except:
        continue


SoftPOSIT: Simultaneous Pose and Correspondence Determination
A Spectral/Spatial CBIR System for Hyperspectral Images
Improved FCM Algorithm for Clustering on Web Usage Mining
Dimensionality in image analysis
Appearance-Based Gaze Estimation Using Visual Saliency
The Henstock Integral and the Black-Scholes Theory of Derivative Asset Pricing
Cheap and Fast – But is it Good? Evaluating Non-Expert Annotations for Natural Language Tasks
Globally Convergent Dual MAP LP Relaxation Solvers using Fenchel-Young Margins
Ultrahigh Dimensional Feature Selection: Beyond The Linear Model
A Secure Semi-fragile Watermarking Algorithm for Image Authentication in the Wavelet Domain of JPEG2000
Clustering for edge-cost minimization (extended abstract)
An Efficient Method for Gradient-Based Adaptation of Hyperparameters in SVM Models
Retrieval of Images Using Mean-Shift and Gaussian Mixtures Based on Weighted Color Histograms
Model-Based Event Detection in Wireless Sensor Networks
View synthesis technique

In [136]:
cursor.fetchall()

[('2957517', 'Tijmen Tieleman'), ('1695689', 'Grace Hinton')]

In [9]:
requests.get("https://www.ncbi.nlm.nih.gov/pmc/utils/idconv/v1.0/.").content

b'<!DOCTYPE html\n\tPUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"\n\t "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="en-US" xml:lang="en-US">\n<head>\n<title>Any ID converter</title>\n<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1" />\n</head>\n<body bgcolor="#F8F8F8">\n<h1>Any ID converter</h1><hr /><form method="get" action="https://www.ncbi.nlm.nih.gov/pmc/utils/idconv/v1.0/" enctype="multipart/form-data"><p><textarea name="ids"  rows="15" cols="60"></textarea> <br /> <br /> Convert: <select name="idtype" >\n<option value="pmcid">pmcid</option>\n<option selected="selected" value="pmid">pmid</option>\n<option value="mid">mid</option>\n<option value="doi">doi</option>\n</select> <br /> Output:&nbsp;&nbsp; <select name="format" >\n<option value="xml">xml</option>\n<option selected="selected" value="html">html</option>\n<option value="json">json</option>\n<option value="csv">csv</option>\n</select> <

In [ ]:
response = requests.get("http://list.didsoft.com/get?email=david.le@inovex.de&pass=m75h35&pid=http3000&showcountry=no").content.decode("utf-8")
proxy_list = response.split("\n")

In [72]:
x = requests.get("https://api.springernature.com/openaccess/jats?q=doi:10.1186/s13326-022-00280-6&api_key=4d550573ce69005bbdb3fc40e70a3b2c").content
#x = requests.get("https://api.springernature.com/openaccess/jats?q=doi:10.1117/12.525746&api_key=4d550573ce69005bbdb3fc40e70a3b2c").content

In [73]:
a = xmltodict.parse(x)

In [74]:
a

{'response': {'apiMessage': 'This XML was provided by Springer Nature',
  'query': 'doi:10.1186/s13326-022-00280-6',
  'apiKey': '4d550573ce69005bbdb3fc40e70a3b2c',
  'result': {'total': '1',
   'start': '1',
   'pageLength': '10',
   'recordsDisplayed': '1'},
  'records': {'article': {'@dtd-version': '1.2',
    '@article-type': 'research-article',
    '@xml:lang': 'en',
    '@xmlns:mml': 'http://www.w3.org/1998/Math/MathML',
    '@xmlns:xlink': 'http://www.w3.org/1999/xlink',
    'front': {'journal-meta': {'journal-id': {'@journal-id-type': 'publisher-id',
       '#text': '13326'},
      'journal-title-group': {'journal-title': 'Journal of Biomedical Semantics',
       'abbrev-journal-title': {'@abbrev-type': 'publisher',
        '#text': 'J Biomed Semant'}},
      'issn': {'@pub-type': 'epub', '#text': '2041-1480'},
      'publisher': {'publisher-name': 'BioMed Central',
       'publisher-loc': 'London'}},
     'article-meta': {'article-id': [{'@pub-id-type': 'publisher-id',
        

In [115]:
contrib_group = a["response"]["records"]["article"]["front"]["article-meta"]["contrib-group"]
all_affiliations = []
for x in contrib_group["contrib"]:
    name = f"{x['name']['given-names']} {x['name']['surname']}"
    affiliation = []
    for aff in x["xref"]:
        if aff["@ref-type"] == "aff":
            ref_id = int(aff["@rid"][-1])
            affiliation.append(contrib_group["aff"][ref_id-1]["institution-wrap"]["institution"]["#text"])

    affiliations = ";".join(affiliation)
    all_affiliations.append((name, affiliations))


In [5]:
cursor.execute("select p.PaperID, p.Title from Papers p where p.DOI is NULL")
all_papers = cursor.fetchall()
len(all_papers)

66965

In [ ]:
for idx,paper in enumerate(all_papers):
    try:
        if idx % 100 == 0:
            print(idx)
        cursor.execute("select distinct a.AuthorID, a.Name from Papers p,authoredBy b,Authors a where p.PaperID=%s and p.PaperID = b.PaperID and b.AuthoredByID = a.AuthorID",[paper[0]])
        authors = cursor.fetchall()

        if not authors:
            cursor.execute("UPDATE Papers p SET p.DOI='None' where p.PaperID=%s",[paper[0]])
            cnx.commit()
            continue
        first_name = authors[0][1].split()[-1]
        doi = research_crawler.extract_doi(paper[1],first_name)
        if doi:
            cursor.execute("UPDATE Papers p SET p.DOI=%s where p.PaperID=%s",[doi,paper[0]])
        else:
            cursor.execute("UPDATE Papers p SET p.DOI='NA' where p.PaperID=%s",[paper[0]])
        cnx.commit()
    except:
        print(paper[1])
        continue

0
Acknowledgements
Pedestrian Detection Inspired by Appearance Constancy and Shape Symmetry
Real Time Fabric Defect Detection System on an Embedded DSP Platform
100
200
